<a href="https://colab.research.google.com/github/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/blob/main/StockSeguridad_dias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer los datos de ventas (Historico)
ventas_df = pd.read_excel(url, sheet_name="Historico")

# Leer los datos de lead time (LeadTime-Dias)
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Seleccionar el primer item (ITEM 1)
item_ventas = ventas_df.iloc[0, 1:]  # Todas las columnas excepto la primera (ITEM)
item_lead_time = lead_time_df.iloc[0, 1:]  # Todas las columnas excepto la primera (ITEM)

# Convertir los valores a numéricos
item_ventas = pd.to_numeric(item_ventas, errors='coerce')
item_lead_time = pd.to_numeric(item_lead_time, errors='coerce')

# Tomar los últimos 18 meses de ventas
ultimos_18_meses = item_ventas[-18:]

# Calcular el promedio de ventas de los últimos 18 meses
promedio_ventas = ultimos_18_meses.mean()

# Calcular la desviación estándar de las ventas de los últimos 18 meses
desviacion_estandar_ventas = ultimos_18_meses.std()

# Calcular el promedio del lead time en días
promedio_lead_time = item_lead_time.mean()

# Convertir el lead time a meses (dividiendo por 30)
L = promedio_lead_time / 30

# R es dado como 0.5 días, convertirlo a meses
R = 0.5 / 30

# Z-score (suponemos Z = 1.645 para un nivel de servicio del 95%)
Z = 1.645

# Aplicar la fórmula del stock de seguridad
SS = Z * desviacion_estandar_ventas * np.sqrt(L + R)

# Calcular el stock de seguridad en días
stock_seguridad_dias = (SS / promedio_ventas) * 30

# Resultados
print(f"Promedio de ventas de los últimos 18 meses: {promedio_ventas:.2f}")
print(f"Desviación estándar de las ventas: {desviacion_estandar_ventas:.2f}")
print(f"Promedio del lead time en días: {promedio_lead_time:.2f}")
print(f"Stock de seguridad en unidades: {SS:.2f}")
print(f"Stock de seguridad en días: {stock_seguridad_dias:.2f}")

Promedio de ventas de los últimos 18 meses: 30696.00
Desviación estándar de las ventas: 4624.90
Promedio del lead time en días: 4.47
Stock de seguridad en unidades: 3096.44
Stock de seguridad en días: 3.03


In [2]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer los datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Asegurarnos de que los nombres de columnas de ITEM sean consistentes
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para almacenar los resultados
resultados = []

# Z-score para 95% de nivel de servicio (puedes cambiarlo si necesitas otro nivel)
Z = 1.645

# R (tiempo de reorden en días)
R_dias = 0.5

# Iterar sobre cada fila (cada ítem) en la hoja de ventas
for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # Obtener las ventas (todas las columnas excepto 'ITEM')
    ventas = row[1:]  # Desde la segunda columna en adelante
    ventas = pd.to_numeric(ventas, errors='coerce')  # Convertir a numérico, NaN si falla

    # Tomar los últimos 18 meses de ventas
    ultimos_18_meses = ventas[-18:]

    # Si hay menos de 2 valores válidos, no se puede calcular
    if ultimos_18_meses.count() < 2:
        print(f"Advertencia: No hay suficientes datos de ventas para {item}")
        stock_seguridad_dias = np.nan
        resultados.append({'ITEM': item, 'Stock_Seguridad_Dias': stock_seguridad_dias})
        continue

    # Calcular promedio y desviación estándar
    promedio_ventas = ultimos_18_meses.mean()
    desviacion_estandar_ventas = ultimos_18_meses.std()

    # Buscar el lead time correspondiente a este ítem
    if item in lead_time_df['ITEM'].values:
        lead_time_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]  # Excluir columna ITEM
        lead_time_row = pd.to_numeric(lead_time_row, errors='coerce')
        promedio_lead_time_dias = lead_time_row.mean()
    else:
        print(f"Advertencia: No se encontró lead time para {item}")
        promedio_lead_time_dias = np.nan

    # Si el lead time es NaN, no se puede calcular
    if pd.isna(promedio_lead_time_dias):
        stock_seguridad_dias = np.nan
        resultados.append({'ITEM': item, 'Stock_Seguridad_Dias': stock_seguridad_dias})
        continue

    # Convertir lead time y R a meses (dividiendo entre 30)
    L = promedio_lead_time_dias / 30
    R = R_dias / 30

    # Calcular stock de seguridad en unidades
    try:
        SS_unidades = Z * desviacion_estandar_ventas * np.sqrt(L + R)
    except:
        SS_unidades = np.nan

    # Convertir a días
    if pd.notna(SS_unidades) and promedio_ventas > 0:
        stock_seguridad_dias = (SS_unidades / promedio_ventas) * 30
    else:
        stock_seguridad_dias = np.nan

    # Agregar al resultado
    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': round(stock_seguridad_dias, 2) if not pd.isna(stock_seguridad_dias) else np.nan
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

# Mostrar el resultado
print("\nStock de Seguridad en Días por Ítem:")
print(resultado_df)

# Opcional: Guardar en Excel
# resultado_df.to_excel("Stock_Seguridad_Dias.xlsx", index=False)


Stock de Seguridad en Días por Ítem:
         ITEM  Stock_Seguridad_Dias
0      ITEM 1                  3.03
1      ITEM 2                  5.03
2      ITEM 3                  2.92
3      ITEM 4                  5.08
4      ITEM 5                  5.27
..        ...                   ...
655  ITEM 656                  3.42
656  ITEM 657                  3.65
657  ITEM 658                  4.05
658  ITEM 659                  4.16
659  ITEM 660                  4.41

[660 rows x 2 columns]


In [3]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Limpiar nombres
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para resultados
resultados = []

# Parámetros
Z = 1.645  # 95% nivel de servicio
R_dias = 0.5
MIN_VALIDOS_PARA_ESTIMAR = 3
WINSOR_LOW = 0.05  # 5% inferior
WINSOR_HIGH = 0.95  # 95% superior

for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # Extraer y convertir ventas
    ventas = pd.to_numeric(row[1:], errors='coerce')

    # --- 🔧 Paso 1: Imputar valores negativos ---
    ventas_imp = ventas.copy()

    negativos = ventas_imp < 0
    for i in negativos[negativos].index:
        pos = list(ventas.index).index(i)
        inicio_ventana = max(0, pos - 6)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= MIN_VALIDOS_PARA_ESTIMAR:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos_positivos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    # Asegurar no negativos
    ventas_imp = ventas_imp.clip(lower=0)

    # --- 🛠️ Paso 2: Winsorización en los últimos 18 meses ---
    ultimos_18_meses = ventas_imp[-18:].copy()

    # Calcular percentiles
    low_percentile = ultimos_18_meses.quantile(WINSOR_LOW)
    high_percentile = ultimos_18_meses.quantile(WINSOR_HIGH)

    # Winsorizar: reemplazar extremos
    ultimos_18_meses_winsor = ultimos_18_meses.clip(lower=low_percentile, upper=high_percentile)

    # --- Calcular promedio y desviación estándar ---
    promedio_ventas = ultimos_18_meses_winsor.mean()

    desv_std = ultimos_18_meses_winsor.std()
    if pd.isna(desv_std) or desv_std == 0:
        desv_std = 0.1 * promedio_ventas if promedio_ventas > 0 else 0.1

    # --- Lead Time ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_row = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_row) > 0:
            promedio_lead_time_dias = lt_row.mean()
        else:
            promedio_lead_time_dias = 5
    else:
        print(f"Advertencia: Sin lead time para {item}. Usando 5 días.")
        promedio_lead_time_dias = 5

    # Convertir a meses
    L = promedio_lead_time_dias / 30
    R = R_dias / 30

    # Calcular Stock de Seguridad en unidades
    try:
        SS_unidades = Z * desv_std * np.sqrt(L + R)
    except:
        SS_unidades = 0.1

    # Convertir a días
    if promedio_ventas > 0:
        stock_seguridad_dias = (SS_unidades / promedio_ventas) * 30
        stock_seguridad_dias = round(stock_seguridad_dias, 2)
    else:
        stock_seguridad_dias = 0.0

    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': stock_seguridad_dias
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

print("\n✅ Stock de Seguridad en Días por Ítem (con imputación + winsorización):")
print(resultado_df)

# Opcional: guardar
# resultado_df.to_excel("Stock_Seguridad_Final_Con_Winsorizacion.xlsx", index=False)

/tmp/ipython-input-2817634041.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24132.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ventas_imp[i] = max(0, estimado)
/tmp/ipython-input-2817634041.py:62: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ultimos_18_meses_winsor = ultimos_18_meses.clip(lower=low_percentile, upper=high_percentile)



✅ Stock de Seguridad en Días por Ítem (con imputación + winsorización):
         ITEM  Stock_Seguridad_Dias
0      ITEM 1                  1.76
1      ITEM 2                  3.28
2      ITEM 3                  1.67
3      ITEM 4                  2.84
4      ITEM 5                  3.47
..        ...                   ...
655  ITEM 656                  2.65
656  ITEM 657                  2.69
657  ITEM 658                  2.73
658  ITEM 659                  2.77
659  ITEM 660                  2.78

[660 rows x 2 columns]


In [4]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Limpiar nombres
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para resultados
resultados = []

# Parámetros
Z = 1.645  # 95% nivel de servicio
R_dias = 0.5
WINSOR_LOW = 0.05   # Percentil 5
WINSOR_HIGH = 0.95  # Percentil 95
MIN_MESES_PARA_CALCULO = 12  # Mínimo meses válidos para calcular SS

for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # Extraer y convertir ventas
    ventas = pd.to_numeric(row[1:], errors='coerce')

    # --- 🔧 Paso 1: Imputar valores negativos ---
    ventas_imp = ventas.copy()

    negativos = ventas_imp < 0
    for i in negativos[negativos].index:
        try:
            pos = list(ventas.index).index(i)
        except:
            continue

        # Ventana de hasta 12 meses antes (no más allá del inicio)
        inicio_ventana = max(0, pos - 12)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            # Usar promedio de todos los valores positivos del ítem
            todos_positivos = ventas[(ventas > 0) & (ventana.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    # Asegurar no negativos
    ventas_imp = ventas_imp.clip(lower=0)

    # --- 🛠️ Paso 2: Tomar los últimos 30 meses ---
    ultimos_30_meses = ventas_imp[-30:]

    # Si no hay suficientes datos válidos
    if len(ultimos_30_meses.dropna()) < MIN_MESES_PARA_CALCULO:
        print(f"Advertencia: Menos de {MIN_MESES_PARA_CALCULO} meses válidos para {item}")
        resultados.append({'ITEM': item, 'Stock_Seguridad_Dias': np.nan})
        continue

    # --- 🎯 Winsorización sobre los últimos 30 meses ---
    low_val = ultimos_30_meses.quantile(WINSOR_LOW)
    high_val = ultimos_30_meses.quantile(WINSOR_HIGH)
    ultimos_30_winsor = ultimos_30_meses.clip(lower=low_val, upper=high_val)

    # --- Calcular promedio y desviación estándar ---
    promedio_ventas = ultimos_30_winsor.mean()

    desviacion = ultimos_30_winsor.std()
    if pd.isna(desviacion) or desviacion == 0:
        desviacion = 0.1 * promedio_ventas if promedio_ventas > 0 else 0.1

    # --- Lead Time ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_row = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_row) > 0:
            promedio_lead_time_dias = lt_row.mean()
        else:
            promedio_lead_time_dias = 5  # valor por defecto
    else:
        print(f"Advertencia: Sin lead time para {item}. Usando 5 días.")
        promedio_lead_time_dias = 5

    # Convertir L y R a meses (30 días por mes)
    L = promedio_lead_time_dias / 30
    R = R_dias / 30

    # --- Calcular Stock de Seguridad en unidades ---
    try:
        SS_unidades = Z * desviacion * np.sqrt(L + R)
    except:
        SS_unidades = 0.1  # valor mínimo de respaldo

    # --- Convertir a días ---
    if promedio_ventas > 0:
        stock_seguridad_dias = (SS_unidades / promedio_ventas) * 30
        stock_seguridad_dias = round(stock_seguridad_dias, 2)
    else:
        stock_seguridad_dias = 0.0  # ítem sin movimiento

    # Guardar resultado
    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': stock_seguridad_dias
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

print("\n✅ Stock de Seguridad en Días por Ítem (últimos 30 meses + imputación + winsorización):")
print(resultado_df)

# Opcional: guardar en Excel
# resultado_df.to_excel("Stock_Seguridad_30Meses_Winsorizado.xlsx", index=False)

/tmp/ipython-input-2623965881.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24132.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ventas_imp[i] = max(0, estimado)
/tmp/ipython-input-2623965881.py:72: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ultimos_30_winsor = ultimos_30_meses.clip(lower=low_val, upper=high_val)
/tmp/ipython-input-2623965881.py:72: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object 


✅ Stock de Seguridad en Días por Ítem (últimos 30 meses + imputación + winsorización):
         ITEM  Stock_Seguridad_Dias
0      ITEM 1                  2.47
1      ITEM 2                  4.30
2      ITEM 3                  2.35
3      ITEM 4                  4.05
4      ITEM 5                  4.49
..        ...                   ...
655  ITEM 656                  3.11
656  ITEM 657                  3.34
657  ITEM 658                  3.57
658  ITEM 659                  3.75
659  ITEM 660                  3.68

[660 rows x 2 columns]


In [5]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Limpiar nombres
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para resultados
resultados = []

# Parámetros
Z = 1.645  # 95% nivel de servicio
R_dias = 0.5  # Tiempo de reorden en días
WINSOR_LOW = 0.05
WINSOR_HIGH = 0.95
MIN_MESES_PARA_CALCULO = 12

for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # Extraer y convertir ventas
    ventas = pd.to_numeric(row[1:], errors='coerce')

    # --- 🔧 Imputar negativos ---
    ventas_imp = ventas.copy()
    negativos = ventas_imp < 0

    for i in negativos[negativos].index:
        try:
            pos = list(ventas.index).index(i)
        except:
            continue
        inicio_ventana = max(0, pos - 12)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos_positivos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    ventas_imp = ventas_imp.clip(lower=0)

    # --- 🛠️ Últimos 30 meses y winsorización ---
    ultimos_30_meses = ventas_imp[-30:]

    if len(ultimos_30_meses.dropna()) < MIN_MESES_PARA_CALCULO:
        resultados.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'SS_Techo_Dias': np.nan,
            'SS_Optimo_Dias': np.nan
        })
        continue

    low_val = ultimos_30_meses.quantile(WINSOR_LOW)
    high_val = ultimos_30_meses.quantile(WINSOR_HIGH)
    ultimos_30_winsor = ultimos_30_meses.clip(lower=low_val, upper=high_val)

    promedio_ventas = ultimos_30_winsor.mean()
    desviacion = ultimos_30_winsor.std()
    if pd.isna(desviacion) or desviacion == 0:
        desviacion = 0.1 * promedio_ventas if promedio_ventas > 0 else 0.1

    # --- Lead Time en días ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_row = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_row) > 0:
            L_dias = lt_row.mean()  # Ya en días
        else:
            L_dias = 5.0
    else:
        print(f"Advertencia: Sin lead time para {item}. Usando 5 días.")
        L_dias = 5.0

    # --- Cálculo del Stock de Seguridad en unidades ---
    L_meses = L_dias / 30
    R_meses = R_dias / 30

    try:
        SS_unidades = Z * desviacion * np.sqrt(L_meses + R_meses)
    except:
        SS_unidades = 0.1

    # Convertir SS a días
    if promedio_ventas > 0:
        SS_dias = (SS_unidades / promedio_ventas) * 30
        SS_dias = round(SS_dias, 2)
    else:
        SS_dias = 0.0

    # --- Stock de Seguridad Techo en Días ---
    SS_techo_dias = SS_dias + L_dias + R_dias
    SS_techo_dias = round(SS_techo_dias, 2)

    # --- Stock Óptimo en Días (corregido) ---
    # Solo se divide para 2 el (L + R), no todo
    SS_optimo_dias = SS_dias + (L_dias + R_dias) / 2
    SS_optimo_dias = round(SS_optimo_dias, 2)

    # Agregar al resultado
    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': SS_dias,
        'SS_Techo_Dias': SS_techo_dias,
        'SS_Optimo_Dias': SS_optimo_dias
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

print("\n✅ Resultados corregidos (SS_Optimo: SS + (L + R)/2):")
print(resultado_df)

# Opcional: guardar
# resultado_df.to_excel("Stock_Seguridad_Final_Corregido.xlsx", index=False)

/tmp/ipython-input-583749106.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24132.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ventas_imp[i] = max(0, estimado)
/tmp/ipython-input-583749106.py:70: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ultimos_30_winsor = ultimos_30_meses.clip(lower=low_val, upper=high_val)
/tmp/ipython-input-583749106.py:70: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dty


✅ Resultados corregidos (SS_Optimo: SS + (L + R)/2):
         ITEM  Stock_Seguridad_Dias  SS_Techo_Dias  SS_Optimo_Dias
0      ITEM 1                  2.47           7.44            4.95
1      ITEM 2                  4.30          17.30           10.80
2      ITEM 3                  2.35           6.22            4.28
3      ITEM 4                  4.05          17.50           10.78
4      ITEM 5                  4.49          15.98           10.23
..        ...                   ...            ...             ...
655  ITEM 656                  3.11          10.01            6.56
656  ITEM 657                  3.34          11.02            7.18
657  ITEM 658                  3.57          12.92            8.25
658  ITEM 659                  3.75          14.34            9.04
659  ITEM 660                  3.68          13.17            8.42

[660 rows x 4 columns]
